In [1]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import special
from scipy import optimize
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
import plotly.express as px
from datetime import date
from glob import glob
from os.path import join, exists
import random
from random import random as rand

from datetime import date

from tqdm import tqdm

In [2]:
## date
today = date.today().strftime("%Y%m%d")

In [3]:
def convert_ID(ID):
    ##################   ##################
    #  1  2  3  4  5 #   #        2       #
    #  6  7  8  9 10 #   #        1       #
    # 11 12 13 14 15 # = # -2 -1  0  1  2 #
    # 16 17 18 19 20 #   #       -1       #
    # 21 22 23 24 25 #   #       -2       #
    ##################   ##################
    x = np.kron(np.ones(5),np.arange(-2,3)).astype(int)
    y = np.kron(np.arange(2,-3,-1),np.ones(5)).astype(int)
    pos = np.array((x[ID-1],y[ID-1]))
    return pos

In [9]:
def calc_hit(behav_data):

    datum = scipy.io.loadmat(behav_data)
    
    ## second per trial
    spt = 5
    ## frame per trial
    fpt = 60*spt
    ## the number of trials per run
    tpr = 96+1
    ## the number of runs
    nrun = 3

    ## target ID
    tmp = datum['targetID'][0]
    targetID = tmp[tmp!=0][:tpr*nrun]

    bx = datum['boxSize'][0][0]
    pos = bx*convert_ID(targetID)
    
    ## 60 Hz * {5 s/trial * (1 trial + 12 trial/block * 8 block)}/Run * 3 Run = 87300
    allXY = datum['allXY']
    
    xFrame, yFrame = np.array([datum['xFrame'].squeeze(), datum['yFrame'].squeeze()]).astype(int)
    
    ## target position
    tmp = np.zeros(allXY.shape)
    for i in range(fpt*tpr*nrun):
        t = np.floor(i/fpt).astype(int)
        tmp[0][i], tmp[1][i] = pos[0][t], pos[1][t]

    ## Is it hit?
    err = allXY - tmp
    hit = np.zeros(fpt*tpr*nrun)
    for i in range(fpt*tpr*nrun):
        hit[i] = abs(err[0][i]) <= bx*.5 and abs(err[1][i]) <= bx*.5
        
    return hit

In [7]:
datum = scipy.io.loadmat('/mnt/ext6/GP/behav_data/GP08-fmri.mat')

In [8]:
datum.keys()

dict_keys(['__header__', '__version__', '__globals__', 'AccRatio', 'FeedbackTime', 'InsideTimeOut', 'Llog', 'Lpath', 'MaxTrials', 'MovementTimeOut', 'PosTarget', 'PreparationTime', 'RetTrials', 'TaskAngle', 'TaskColor', 'TaskPos', 'TitleDisplayTime', 'Tlog', 'TotalLearnTrialTime', 'TotalTrials', 'Tpath', 'cm2px', 'diacm', 'diameter', 'distcm', 'isflipped', 'nRun', 'nTask', 'nTrialperRun', 'xFrame', 'yFrame', 'subjID', 'testType', 'LogDir', 'Clog', 'K', 'weight', 'sumw', 'Gx', 'Gy', 'X0', 'Y0', 'res', 'fontSize', 'key', 't0', 'gridDraw_trial', 'targetID', 'nTrial', 'glove', 'allData', 'cnt', 'LearnTrialStartTime', 'data', 'rate', 'CaliDuration', 'mData', 'U', 'S', 'A', 'strSubj', 'dataTemp', 'XY', 'boxSize', 'pos', 'ID_reached', 'allXY', 'ID', 'mouseSize', 'mouseThick', 'PreDuration', 'T', 'nSampleTrial', 'nSample', 'fb', 'preData', 'tpos', 'cnt_hit', '__function_workspace__'])

In [10]:
hit = calc_hit('/mnt/ext6/GP/behav_data/GP08-fmri.mat')

In [11]:
hit.shape

(87300,)